## Initial Merge of raw databases

In [3]:
import pandas as pd
import os

folder_path = r'..\data\raw'
file1 = os.path.join(folder_path, 'or1.xlsx')
file2 = os.path.join(folder_path, 'or2.xlsx')

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

merged_df = pd.concat([df1, df2], ignore_index=True)

output_path_csv = os.path.join(folder_path, 'merged_ironman_data.csv')
merged_df.to_csv(output_path_csv, index=False)

print(f"Success! Saved as CSV: {output_path_csv}")

Success! Saved as CSV: ..\data\raw\merged_ironman_data.csv


## First data exploration

In [4]:
print("Column Names:", merged_df.columns.tolist())

print("\nMissing Values per Column:")
print(merged_df.isnull().sum())

print("\nSummary Statistics:")
print(merged_df.describe(include='all'))

Column Names: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'Invoice', 'Price', 'Customer ID']

Missing Values per Column:
InvoiceNo      525461
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
UnitPrice      525461
CustomerID     660541
Country             0
Invoice        541909
Price          541909
Customer ID    649836
dtype: int64

Summary Statistics:
        InvoiceNo StockCode                         Description      Quantity  \
count    541909.0   1067370                             1062988  1.067370e+06   
unique    25900.0      5305                                5698           NaN   
top      573585.0    85123A  WHITE HANGING HEART T-LIGHT HOLDER           NaN   
freq       1114.0      5829                                5918           NaN   
mean          NaN       NaN                                 NaN  9.938907e+00   
min           NaN       NaN                           

## Data Cleaning

In [ ]:
merged_df['InvoiceNo'] = merged_df['InvoiceNo'].fillna(merged_df['Invoice'])
merged_df['UnitPrice'] = merged_df['UnitPrice'].fillna(merged_df['Price'])
merged_df['CustomerID'] = merged_df['CustomerID'].fillna(merged_df['Customer ID'])

cols_to_drop = ['Invoice', 'Price', 'Customer ID']
merged_df.drop(columns=cols_to_drop, inplace=True)

# Handle Missing Values
merged_df['Description'] = merged_df['Description'].fillna('Unknown')

# CustomerID
merged_df['CustomerID'] = merged_df['CustomerID'].fillna(0).astype(int)

# Data Type Correction
merged_df['InvoiceDate'] = pd.to_datetime(merged_df['InvoiceDate'])

print("Cleaned Data Overview:")
print(merged_df.info())

## Features ready for ML model?

In [ ]:
print(f"Unique Customers: {merged_df['CustomerID'].nunique()}")
print(f"Unique Products: {merged_df['StockCode'].nunique()}")

import numpy as np
print("\nAny infinite values?")
print(np.isinf(merged_df.select_dtypes(include=np.number)).any())

## Saving cleaned DF for Project Iron Man

In [ ]:
import pandas as pd
import os

# Reload the raw merged data

# Safe Merge and Type Conversion (Same as before)
if 'Invoice' in merged_df.columns:
    merged_df['InvoiceNo'] = merged_df['InvoiceNo'].fillna(merged_df['Invoice'])
if 'Price' in merged_df.columns:
    merged_df['UnitPrice'] = merged_df['UnitPrice'].fillna(merged_df['Price'])
if 'Customer ID' in merged_df.columns:
    merged_df['CustomerID'] = merged_df['CustomerID'].fillna(merged_df['Customer ID'])

merged_df['InvoiceNo'] = merged_df['InvoiceNo'].astype(str)
merged_df['CustomerID'] = merged_df['CustomerID'].fillna(0)

# THE CHANGE: Smart Filtering
cleaned_df = merged_df[merged_df['UnitPrice'] > 0].copy()

# Create an 'Is_Return' flag for easier ML processing later
cleaned_df['Is_Return'] = cleaned_df['Quantity'] < 0

print("New Cleaning Results:")
print(f"Total Transactions: {len(cleaned_df)}")
print(f"Number of Returns Preserved: {cleaned_df['Is_Return'].sum()}")

# Overwrite your clean parquet file
output_path = r'C:\Users\ASUS\Desktop\courses\projects\iron-man\data\clean\ironman.parquet'
cleaned_df.to_parquet(output_path, index=False)

In [ ]:
## Redoing data cleaning to not remove neg sales

In [ ]:
import os

folder_path = r'C:\Users\ASUS\Desktop\courses\projects\iron-man\data\raw'

if 'Invoice' in merged_df.columns:
    merged_df['InvoiceNo'] = merged_df['InvoiceNo'].fillna(merged_df['Invoice'])

if 'Price' in merged_df.columns:
    merged_df['UnitPrice'] = merged_df['UnitPrice'].fillna(merged_df['Price'])

if 'Customer ID' in merged_df.columns:
    merged_df['CustomerID'] = merged_df['CustomerID'].fillna(merged_df['Customer ID'])

cols_to_drop = ['Invoice', 'Price', 'Customer ID']
existing_cols_to_drop = [c for c in cols_to_drop if c in merged_df.columns]
merged_df.drop(columns=existing_cols_to_drop, inplace=True)

merged_df['InvoiceNo'] = merged_df['InvoiceNo'].astype(str)
merged_df['StockCode'] = merged_df['StockCode'].astype(str)
merged_df['InvoiceDate'] = pd.to_datetime(merged_df['InvoiceDate'])

merged_df = merged_df[(merged_df['Quantity'] > 0) & (merged_df['UnitPrice'] > 0)]

output_parquet = os.path.join(folder_path, 'ironman2.parquet')
merged_df.to_parquet(output_parquet, index=False)

print(f"File cleaned and saved successfully to: {output_parquet}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# LOAD & FIX THE DATA
raw_file = r'C:\Users\ASUS\Desktop\courses\projects\iron-man\data\raw\merged_ironman_data.csv'

print(f"Reading file: {raw_file}...")
# encoding='ISO-8859-1' helps if the CSV has weird currency symbols like £
df = pd.read_csv(raw_file, encoding='ISO-8859-1')

# fill the missing ID
df['CustomerID'] = df['CustomerID'].fillna(0)

# filter out items with 0 price (junk), BUT i keep negative Quantities (returns)
df = df[df['UnitPrice'] > 0]

# Calculate TotalPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Save this version so we stop losing returns!
clean_path = r'C:\Users\ASUS\Desktop\courses\projects\iron-man\data\clean\ironman2.parquet'
df.to_parquet(clean_path, index=False)
print(f"Saved fixed file to: {clean_path}")

# Check if I actually have returns now
num_returns = len(df[df['Quantity'] < 0])
print(f"SUCCESS! Found {num_returns} return rows.")


# --- PART 2: THE GRAPH THAT WAS BLANK ---
if num_returns > 0:
    # Clean descriptions
    df['Description'] = df['Description'].astype(str).str.strip()

    # Get Sales (Sold)
    sales = df[df['Quantity'] > 0].groupby('Description')['Quantity'].sum().reset_index()
    sales.columns = ['Description', 'Sold']

    #Get Returns
    rets = df[df['Quantity'] < 0].copy()
    rets['Qty_Abs'] = rets['Quantity'].abs()
    returns = rets.groupby('Description')['Qty_Abs'].sum().reset_index()
    returns.columns = ['Description', 'Returned']

    # merge
    stats = pd.merge(sales, returns, on='Description', how='inner')
    
    # Math time
    stats['Return_Prob'] = (stats['Returned'] / stats['Sold']) * 100
    
    # Filter: Only show items sold > 50 times (to ignore rare stuff)
    top_junk = stats[stats['Sold'] > 50].sort_values('Return_Prob', ascending=False).head(10)

    # Plot
    print("\nGenerating the Headaches Graph...")
    plt.figure(figsize=(10, 6))
    sns.barplot(data=top_junk, x='Return_Prob', y='Description', palette='Reds_r')
    plt.title('Top 10 Products by Return Probability (Items sold > 50 times)')
    plt.xlabel('Return Probability (%)')
    plt.show()

    # 8. Print the list
    print("\nTop risky items:")
    print(top_junk[['Description', 'Sold', 'Returned', 'Return_Prob']])

else:
    print("0 returns")

## Saving it into a SQL database

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

# 1. Load credentials
load_dotenv()
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

# 2. Load Parquet
file_path = r"C:\Users\ASUS\Desktop\courses\projects\iron-man\data\clean\ironman2.parquet"
df = pd.read_parquet(file_path)

# Direct Connection
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{db_name}')

print(f"Connecting to {db_name}...")

try:
    # Upload Data
    # chunksize helps if the file is large, preventing memory overflows
    print("Uploading... this may take a moment.")
    df.to_sql('transactions', con=engine, if_exists='replace', index=False, chunksize=1000)
    print("Success! Data is now in MySQL.")
    
except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Always close the engine to prevent future locks
    engine.dispose()

Connecting to ironman...
Uploading... this may take a moment.
Success! Data is now in MySQL.
